In [2]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

# --- Import classic ISR spectrum code (Marco Milla) ---
sys.path.append(os.path.join(os.getcwd(), 'EEApproach'))
import plasma_parameters as plasma_param
import physical_constants as phys_cons
from isr_spectrum import isrSpectrum

# --- Import ISRPlottingSpectrSite-main code (Chirag's) ---
from spectratestingmodified_chirag_edit import (
    calculate_wavenumber_components,
    calculate_thermal_velocity,
    calculate_cyclotron_frequency,
    calculate_average_gyroradius,
    calculate_debye_length,
    calculate_alpha,
    calculate_omega_values,
    calculate_collisional_term,
    calculate_modified_distribution,
    calculate_electric_susceptibility,
    calcSpectra
)

def run_marco_spectrum(Ne, Bm, Te, Ti, ion_mass, ion_comp, N, fs, frad, aspdeg, modgy, modnu):
    plasma = plasma_param.Plasma(Ne, Bm, Te, Ti, ion_mass, ion_comp)
    lambdaB = phys_cons.c / frad / 2
    isr_spc = isrSpectrum(N, fs, lambdaB, aspdeg, plasma, modgy=modgy, modnu=modnu)
    return isr_spc.f / 1E3, isr_spc.spc

def run_chirag_spectrum(ne_, B, Te, Ti, mi, radar_freq, theta):
    nu_i = 1e-7
    nu_e = 1e-7
    m_e = 9.11e-31
    epsilon_0 = 8.854187817e-12
    kB = 1.380649e-23
    e = 1.602e-19
    n_terms = 2000

    c_light = 3e8
    lambda_wavelength = c_light / radar_freq

    k_total, k_parallel, k_perpendicular = calculate_wavenumber_components(lambda_wavelength, theta)
    vth_e = calculate_thermal_velocity(kB, Te, m_e)
    vth_i = calculate_thermal_velocity(kB, Ti, mi)
    Oc_e = calculate_cyclotron_frequency(1, e, B, m_e)
    Oc_i = calculate_cyclotron_frequency(1, e, B, mi)
    rho_e = calculate_average_gyroradius(vth_e, Oc_e)
    rho_i = calculate_average_gyroradius(vth_i, Oc_i)
    lambda_De = calculate_debye_length(Te, ne_, epsilon_0, kB, e)
    alpha = calculate_alpha(k_total, lambda_De)
    omega = calculate_omega_values(k_total, vth_i)

    U_i = calculate_collisional_term(nu_i, k_parallel, vth_i, k_perpendicular, rho_i, n_terms, omega, Oc_i)
    U_e = calculate_collisional_term(nu_e, k_parallel, vth_e, k_perpendicular, rho_e, 20, omega, Oc_e)
    M_i = calculate_modified_distribution(omega, k_parallel, k_perpendicular, vth_i, n_terms, rho_i, Oc_i, nu_i, U_i)
    M_e = calculate_modified_distribution(omega, k_parallel, k_perpendicular, vth_e, 20, rho_e, Oc_e, nu_e, U_e)
    chi_i = calculate_electric_susceptibility(omega, k_parallel, k_perpendicular, vth_i, n_terms, rho_i, Oc_i, nu_i, alpha, U_i, Te, Ti)
    chi_e = calculate_electric_susceptibility(omega, k_parallel, k_perpendicular, vth_e, 20, rho_e, Oc_e, nu_e, alpha, U_e, Te, Te)
    spectrum = calcSpectra(M_i, M_e, chi_i, chi_e)

    freq_kHz = omega / (2 * np.pi * 1e3)
    return freq_kHz, spectrum

def align_spectra(f1, spc1, f2, spc2):
    f_min = max(np.min(f1), np.min(f2))
    f_max = min(np.max(f1), np.max(f2))
    num = min(len(f1), len(f2))
    f_common = np.linspace(f_min, f_max, num)
    spc1_interp = np.interp(f_common, f1, spc1)
    spc2_interp = np.interp(f_common, f2, spc2)
    return f_common, spc1_interp, spc2_interp

def plot_all(Ne, Bm, Te, Ti, ion_mass, ion_comp, N, fs, frad, aspdeg, modgy, modnu, B, mi, radar_freq, theta):
    f_marco, spc_marco = run_marco_spectrum(Ne, Bm, Te, Ti, ion_mass, ion_comp, N, fs, frad, aspdeg, modgy, modnu)
    f_chirag, spc_chirag = run_chirag_spectrum(Ne, B, Te, Ti, mi, radar_freq, theta)
    f_common, spc_marco_interp, spc_chirag_interp = align_spectra(f_marco, spc_marco, f_chirag, spc_chirag)

    # Overlayed spectra
    plt.figure(figsize=(12, 6))
    plt.plot(f_common, 10 * np.log10(spc_marco_interp), label="Marco's isrSpectrum (log dB)", alpha=0.8)
    plt.plot(f_common, 10 * np.log10(spc_chirag_interp), label="Chirag's ISR Site (log dB)", alpha=0.8)
    plt.grid()
    plt.xlim([-6e3, 6e3])
    plt.xlabel('Frequency [kHz]')
    plt.ylabel('Spectra [dB]')
    plt.title('ISR Spectrum Comparison: EE Approach vs. ISR Site')
    plt.legend()
    plt.tight_layout()
    plt.show()

    # Difference plot
    diff = 10 * np.log10(spc_marco_interp) - 10 * np.log10(spc_chirag_interp)
    plt.figure(figsize=(12, 4))
    plt.plot(f_common, diff, label='Difference (Marco - Chirag) [dB]')
    plt.grid()
    plt.xlim([-6e3, 6e3])
    plt.xlabel('Frequency [kHz]')
    plt.ylabel('Difference [dB]')
    plt.title('Spectra Difference')
    plt.legend()
    plt.tight_layout()
    plt.show()

    # Ratio plot
    ratio = spc_marco_interp / spc_chirag_interp
    ratio_dB = 10 * np.log10(ratio)
    plt.figure(figsize=(12, 4))
    plt.plot(f_common, ratio_dB, label='Ratio (Marco/Chirag) [dB]')
    plt.grid()
    plt.xlim([-6e3, 6e3])
    plt.xlabel('Frequency [kHz]')
    plt.ylabel('Ratio [dB]')
    plt.title('Spectra Ratio')
    plt.legend()
    plt.tight_layout()
    plt.show()

# --- Widgets for interactive controls ---
Ne_widget = widgets.FloatText(value=2e11, description='Electron Density (m⁻³)')
Bm_widget = widgets.FloatText(value=36e-6, description='Magnetic Field (T)')
Te_widget = widgets.FloatText(value=500, description='Electron Temp (K)')
Ti_widget = widgets.FloatText(value=500, description='Ion Temp (K)')
ion_mass_widget = widgets.FloatText(value=16, description='Ion Mass (amu)')
ion_comp_widget = widgets.FloatText(value=1, description='Ion Comp')
N_widget = widgets.IntText(value=100000, description='FFT Points (N)')
fs_widget = widgets.FloatText(value=12e6, description='Sampling Freq (Hz)')
frad_widget = widgets.FloatText(value=440e6, description='Radar Freq (Hz)')
aspdeg_widget = widgets.FloatText(value=30, description='Aspect Angle (deg)')
modgy_widget = widgets.IntSlider(value=1, min=0, max=2, step=1, description='modgy')
modnu_widget = widgets.IntSlider(value=2, min=0, max=2, step=1, description='modnu')
B_widget = widgets.FloatText(value=3.6e-5, description='B (T, Chirag)')
mi_widget = widgets.FloatText(value=2.65686e-26, description='Ion Mass (kg, Chirag)')
radar_freq_widget = widgets.FloatText(value=440e6, description='Radar Freq (Hz, Chirag)')
theta_widget = widgets.FloatText(value=30, description='Aspect Angle (deg, Chirag)')

ui = widgets.VBox([
    Ne_widget, Bm_widget, Te_widget, Ti_widget, ion_mass_widget, ion_comp_widget,
    N_widget, fs_widget, frad_widget, aspdeg_widget, modgy_widget, modnu_widget,
    B_widget, mi_widget, radar_freq_widget, theta_widget
])

def on_button_clicked(b):
    clear_output(wait=True)
    display(ui, plot_button)
    plot_all(
        Ne_widget.value, Bm_widget.value, Te_widget.value, Ti_widget.value,
        ion_mass_widget.value, ion_comp_widget.value, N_widget.value, fs_widget.value,
        frad_widget.value, aspdeg_widget.value, modgy_widget.value, modnu_widget.value,
        B_widget.value, mi_widget.value, radar_freq_widget.value, theta_widget.value
    )

plot_button = widgets.Button(description="Plot Spectra")
plot_button.on_click(on_button_clicked)

display(ui, plot_button)

ModuleNotFoundError: No module named 'plasma_parameters'